# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [30]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [31]:
# your code here
orders = pd.read_csv('Orders\Orders.csv')
display(orders.head())

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [32]:
## Sub Problem 1:

# I'll use '.groupby' to aggregate the total amount spent by each customer, then I'll save it in 'total_amount'
total_amount = orders.groupby(['CustomerID']).agg({'amount_spent' : 'sum'}).reset_index()

total_amount


,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [33]:
## Sub Problem 2:

# With '.pd.qcut()' I'll divide the data into quantiles (more specifically, into ventiles), and create an extra column
labels = ['5th','10th','15th','20th','25th','30th','35th','40th','45th','50th','55th','60th','65th','70th','75th','80th','85th','90th','95th']
total_amount['customer_bins'] = pd.qcut(total_amount['amount_spent'], 19, labels=labels)
total_amount

,CustomerID,amount_spent,customer_bins
0,12346,77183.60,95th
1,12347,4310.00,90th
2,12348,1797.24,75th
3,12349,1757.55,75th
4,12350,334.40,30th
...,...,...,...
4334,18280,180.60,15th
4335,18281,80.82,5th
4336,18282,178.05,15th
4337,18283,2094.88,80th


In [34]:
## Sub Problem 3:

# To label each customer as VIP or Preferred, I'll use the apply function, I'm also creating a 'Basic' label to avoid Null values
def topcustomers(row):
    if row['customer_bins'] == '95th':
        return 'VIP'
    elif (row['customer_bins'] == '75th')|(row['customer_bins'] == '80th')|(row['customer_bins'] == '90th'):
        return 'Preferred'
    else:
        return 'Basic'

total_amount['category'] = total_amount.apply(topcustomers, axis=1)
total_amount

,CustomerID,amount_spent,customer_bins,category
0,12346,77183.60,95th,VIP
1,12347,4310.00,90th,Preferred
2,12348,1797.24,75th,Preferred
3,12349,1757.55,75th,Preferred
4,12350,334.40,30th,Basic
...,...,...,...,...
4334,18280,180.60,15th,Basic
4335,18281,80.82,5th,Basic
4336,18282,178.05,15th,Basic
4337,18283,2094.88,80th,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [35]:
# your code here

# First, I'll add the column country from the 'orders' dataframe
total_amount = pd.merge(
    left=total_amount,
    right=orders[['CustomerID','Country']],
    how='inner',
    on='CustomerID'
)

In [40]:
# Then, I'll create a condition to filter by 'VIP' customers and aggregate the data to check which country has the biggest number.
condition = total_amount['category'] == 'VIP'
onlyVIP = total_amount[condition]

mostVIP = onlyVIP.groupby(['Country']).agg({'category':'count'}).sort_values(by='category', ascending=False).reset_index()

print('The country with the most VIP customers is',mostVIP['Country'][0]+', with',mostVIP['category'][0],'VIP customers.')

The country with the most VIP customers is United Kingdom, with 87302 VIP customers.


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [44]:
condition2 = (total_amount['category'] == 'VIP')|(total_amount['category'] == 'Preferred')
combined = total_amount[condition2]

bestcustomers = combined.groupby(['Country']).agg({'category':'count'}).sort_values(by='category', ascending=False).reset_index()

print('The country with the most VIP customers is also',bestcustomers['Country'][0]+', with',bestcustomers['category'][0],'VIP and Preferred customers.')

The country with the most VIP customers is also United Kingdom, with 188053 VIP and Preferred customers.
